In [1]:
######Libraries
import cudf
import pandas as pd
import numpy as np
from numba import cuda
import torch
import os

In [2]:
#####Setting GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
'''Version(s)'''
print("pandas version:",pd.__version__)
print("cudf version:",cudf.__version__)
print("numpy version:",np.__version__)
print("cuda version:",torch.version.cuda)

pandas version: 0.25.3
cudf version: 0.13.0+0.ga2804c3.dirty
numpy version: 1.18.1
cuda version: 9.0.176


In [4]:
'''Read Files
cdf_file is the cudf file i.e. on GPU
pd_file is the pandas variant i.e. on CPU
Note cudf doesn't support read/write from/to excel,pickle files'''
import io
%time cdf_file=cudf.read_csv("online_retail.csv")
%time pd_file=pd.read_csv("online_retail.csv")

/home/shravan/anaconda3/lib/python3.7/site-packages/fsspec/implementations/local.py:33: FutureWarning: The default value of auto_mkdir=True has been deprecated and will be changed to auto_mkdir=False by default in a future release.
  FutureWarning,


CPU times: user 565 ms, sys: 379 ms, total: 944 ms
Wall time: 978 ms
CPU times: user 345 ms, sys: 36.3 ms, total: 382 ms
Wall time: 380 ms


In [5]:
'''Memory consumption for both cudf and pandas'''
print("Memory consumed by the CuDF:",(sum(cdf_file.memory_usage()))/1e9,"GB")
print("Memory consumed by the pandas frame:",(sum(pd_file.memory_usage()))/1e9,"GB")

Memory consumed by the CuDF: 0.033826688 GB
Memory consumed by the pandas frame: 0.033629632 GB


In [6]:
'''Sub-setting Data-1
Using loc'''
cdf_file_subset=cdf_file.loc[1:1000]
pd_file_subset=pd_file.loc[1:1000]

In [7]:
'''Sub-setting Data-2
Using iloc'''
cdf_file_subset=cdf_file.iloc[1:5]
pd_file_subset=pd_file.iloc[1:5]

In [8]:
####Note the cudf querying runs on latest cudf i.e. 0.6.1.
cdf_file_query=cdf_file[cdf_file.Price>10]
pd_file_query=pd_file[pd_file.Price>10]

In [9]:
'''Frequency counts
1. Using value_counts
2. Using group by as a substitute for value_count method

CuDF doesn't support value_counts on a string column prior to 0.8.
Similarly other standard methods available in pandas for string data-types are unsupported unless we make us of nvstrings package
CuDF Documentation:https://rapidsai.github.io/projects/cudf/en/latest/api.html
'''
%time pd_count = pd_file['Country'].value_counts()
%time pd_count1 = pd_file.groupby(['Country'])['StockCode'].count() 
%time cudf_count = cdf_file['Country'].value_counts()
####Note the cudf querying runs on latest cudf i.e. 0.6.1.
%time cudf_count = cdf_file.groupby(['Country'])['StockCode'].count()

CPU times: user 29.2 ms, sys: 335 µs, total: 29.5 ms
Wall time: 29 ms
CPU times: user 31.1 ms, sys: 0 ns, total: 31.1 ms
Wall time: 30.7 ms
CPU times: user 18.4 ms, sys: 558 µs, total: 19 ms
Wall time: 18.7 ms
CPU times: user 15.2 ms, sys: 0 ns, total: 15.2 ms
Wall time: 15 ms


In [10]:
'''Sorting data'''
cdf_file=cdf_file.sort_values(by=['Invoice','StockCode'],ascending=True)
pd_file=pd_file.sort_values(by=['Invoice','StockCode'],ascending=True)

In [11]:
'''Extending frames'''
cdf_file1=cudf.concat([cdf_file,cdf_file],ignore_index=True)
pd_file1=pd.concat([pd_file,pd_file],ignore_index=True)

In [12]:
'''Merging frames'''
########Note to join frames both frames need to have the same reference and id names prior to 0.6
cdf_file1=cdf_file.merge(cdf_file[['Invoice','StockCode','Quantity']].rename(columns={'Quantity':'Qty_y','Invoice':'inv','StockCode':"stk"}),left_on=['Invoice','StockCode'],right_on=['inv','stk'],how="inner")
#cdf_file1=cdf_file.merge(cdf_file[['Invoice','StockCode','Quantity']].rename(columns={'Quantity':'Qty_y'}),on=['Invoice','StockCode'],how="inner")
pd_file1=pd_file.merge(pd_file[['Invoice','StockCode','Quantity']].rename(columns={'Quantity':'Qty_y','Invoice':'inv','StockCode':"stk"}),left_on=['Invoice','StockCode'],right_on=['inv','stk'],how="inner")

In [13]:
'''Computation of invoice,item level net price
Approach-1:Vectorized approach
'''
%time cdf_file['Net_Price']=cdf_file.Quantity*cdf_file.Price
%time pd_file['Net_Price']=pd_file.Quantity*pd_file.Price

CPU times: user 177 ms, sys: 355 µs, total: 177 ms
Wall time: 179 ms
CPU times: user 6.76 ms, sys: 5.36 ms, total: 12.1 ms
Wall time: 6.73 ms


In [14]:
'''Computation of invoice,item level net price
Approach-2:Row-wise
apply chunk:incols i.e. columns required as input,outcols i.e. output generated post processing.
Note incols and outcols to be of int/float/datetime arrays.chunks is the number of rows to be allotted to each block and tpb is threads per block.
CUDA works on the principle of threads and not cores.The looping construct is automatically unrolled to the parallel 
variant by the compiler.
'''
def set_net_item_price_cudf(Quantity, Price, out):
    for i, (x, y) in enumerate(zip(Quantity,Price)):
        out[i] = x * y
def set_net_item_price_pd(Quantity, Price):
    return(Quantity*Price)
%time outdf_cudf=cdf_file.apply_chunks(set_net_item_price_cudf,incols=['Quantity', 'Price'],outcols=dict(out=np.float64),kwargs=dict(),chunks=16,tpb=10)
outdf_pandas=pd_file
%time outdf_pandas['out']=outdf_pandas.apply(lambda x: set_net_item_price_pd(Quantity=x['Quantity'],Price=x['Price']),axis=1)

CPU times: user 203 ms, sys: 3.36 ms, total: 207 ms
Wall time: 209 ms
CPU times: user 6.45 s, sys: 37.1 ms, total: 6.48 s
Wall time: 6.53 s


In [15]:
%%time
######The current cudf vaersion doesn't support drop_duplicates.Hence the current code is not executable.The code should run on cudf version>0.5
def create_master(series,name_to_refer,name_to_id):
    master=cudf.DataFrame()
    master[name_to_refer]=series
    master=master.drop_duplicates()
    master[name_to_id]=np.arange(0,(len(master)))
    return(master)
invoice_master=create_master(cdf_file.Invoice,name_to_refer="Invoice",name_to_id="Invoice_ID")
stock_master=create_master(cdf_file.StockCode,name_to_refer="StockCode",name_to_id="StockCode_ID")
cdf_file1=cdf_file[['Invoice','StockCode','Quantity','Price']]
cdf_file1=cdf_file1.merge(invoice_master,how="left")
cdf_file1=cdf_file1.merge(stock_master,how="left")

CPU times: user 19.7 ms, sys: 3.99 ms, total: 23.6 ms
Wall time: 22.9 ms


In [16]:
%%time
'''Computation of invoice,item level net price
Approach-3:Group-Wise
apply_grouped:incols i.e. columns required as input,outcols i.e. output generated post processing.
Note incols and outcols to be of int/float/datetime arrays.In principle the chunks is now a variable determined on the basis of # of instances for the group.Useful for row-wise operations on groups.
'''
def grouped_summary(StockCode_ID,Price,Quantity,Net_price):
    for i in range(cuda.threadIdx.x, len(StockCode_ID), cuda.blockDim.x):
        Net_price[i] = Price[i] * Quantity[i]
cdf_file2=cdf_file1[['StockCode_ID','Invoice_ID','Quantity','Price']].groupby(['StockCode_ID'], method='cudf').apply_grouped(grouped_summary,incols=['StockCode_ID','Price','Quantity'],outcols={'Net_price': np.float64},tpb=600)
cdf_file2=cdf_file2.merge(stock_master,how="left")
cdf_file2=cdf_file2.merge(invoice_master,how="left")

CPU times: user 237 ms, sys: 8.09 ms, total: 245 ms
Wall time: 247 ms


In [17]:
print("apply_chunks:cudf")
print(outdf_cudf[outdf_cudf.Invoice=="489434"][['Invoice','StockCode','out']].sort_values(by=["StockCode","Invoice"]))
print("apply_grouped:cudf")
print(cdf_file2[cdf_file2.Invoice=="489434"][['Invoice','StockCode','Net_price']].sort_values(by=["StockCode"]))
print("apply:pandas")
print(outdf_pandas[outdf_pandas.Invoice=="489434"][['Invoice','StockCode','out']].sort_values(by=["StockCode","Invoice"]))

apply_chunks:cudf
  Invoice StockCode    out
4  489434     21232   30.0
7  489434     21523   59.5
6  489434     21871   30.0
3  489434     22041  100.8
5  489434     22064   39.6
1  489434    79323P   81.0
2  489434    79323W   81.0
0  489434     85048   83.4
apply_grouped:cudf
       Invoice StockCode  Net_price
91060   489434     21232       30.0
134364  489434     21523       59.5
190931  489434     21871       30.0
197024  489434     22041      100.8
209554  489434     22064       39.6
412114  489434    79323P       81.0
435094  489434    79323W       81.0
466269  489434     85048       83.4
apply:pandas
  Invoice StockCode    out
4  489434     21232   30.0
7  489434     21523   59.5
6  489434     21871   30.0
3  489434     22041  100.8
5  489434     22064   39.6
1  489434    79323P   81.0
2  489434    79323W   81.0
0  489434     85048   83.4


In [18]:
'''Describe doesn't work with cudf if string columns are present in versions prior to 0.6.1.
'''

print("cudf_describe")
print(cdf_file.describe())
print("\n\npandas describe")
print(pd_file.describe())

cudf_describe
            Quantity          Price    Customer ID      Net_Price
count  525461.000000  525461.000000  417534.000000  525461.000000
mean       10.337667       4.688834   15360.645478      18.154506
std       107.424110     146.126914    1680.811316     160.333083
min     -9600.000000  -53594.360000   12346.000000  -53594.360000
25%         1.000000       1.250000   13983.000000       3.750000
50%         3.000000       2.100000   15311.000000       9.950000
75%        10.000000       4.210000   16799.000000      17.700000
max     19152.000000   25111.090000   18287.000000   25111.090000


pandas describe
            Quantity          Price    Customer ID      Net_Price  \
count  525461.000000  525461.000000  417534.000000  525461.000000   
mean       10.337667       4.688834   15360.645478      18.154506   
std       107.424110     146.126914    1680.811316     160.333083   
min     -9600.000000  -53594.360000   12346.000000  -53594.360000   
25%         1.000000       1.

In [19]:
'''Group-By on frames'''
cdf_group_by=cdf_file.groupby('Country',as_index=False).agg({'Price':['sum','min','max'],'Quantity' : ['sum', 'max','min'],'Net_Price':['sum','max','min']})
print(cdf_group_by.head())
pd_group_by=pd_file.groupby('Country').agg({'Price' : ['sum', 'max','min'], 'Quantity' : ['sum', 'max','min'],'Net_Price':['sum','max','min']})

     Country    Price                Quantity           Net_Price           \
                  sum   min      max      sum   max min       sum      max   
0  Australia  4056.32  0.29   662.25    20053   480 -24  30051.80   876.00   
1    Austria  2482.80  0.12   130.00     6479   120 -36  13023.28   297.00   
2    Bahrain   352.92  0.42    14.95     1015    96 -10   2313.15   187.20   
3    Belgium  7226.75  0.00  1508.65    11980   120 -30  22663.53  1508.65   
4    Bermuda    84.70  0.21    12.75     2798  1152   2   1253.14   241.92   

            
       min  
0  -662.25  
1  -130.00  
2   -42.50  
3 -1508.65  
4    10.20  


In [20]:
'''Categories support is provided from pandas'''
pd_file2=pd_file.copy()
pd_file2['Country_Cat']=pd_file2.Country.copy()
pd_file2['Country_Cat']=pd_file2.Country_Cat.astype("category")
#print(pd_file2.columns)

cdf_file2=cudf.DataFrame.from_pandas(pd_file2.copy())

type(cdf_file2.Country_Cat.cat)
print("Categorical Labels")
print(cdf_file2.Country_Cat.cat.categories)
print(cdf_file2.Country_Cat.cat.codes)

Categorical Labels
StringIndex(['Australia' 'Austria' 'Bahrain' 'Belgium' 'Bermuda' 'Brazil' 'Canada'
 'Channel Islands' 'Cyprus' 'Denmark' 'EIRE' 'Finland' 'France' 'Germany'
 'Greece' 'Hong Kong' 'Iceland' 'Israel' 'Italy' 'Japan' 'Korea' 'Lebanon'
 'Lithuania' 'Malta' 'Netherlands' 'Nigeria' 'Norway' 'Poland' 'Portugal'
 'RSA' 'Singapore' 'Spain' 'Sweden' 'Switzerland' 'Thailand' 'USA'
 'United Arab Emirates' 'United Kingdom' 'Unspecified' 'West Indies'], dtype='object')
0         37
1         37
2         37
3         37
4         37
          ..
525456    13
525457    37
525458    37
525459    37
525460    37
Length: 525461, dtype: int8


In [21]:
'''String Functionality
CuDF supports only nvstrings for string based maipulations.
The functionality for strings is quite similar to re based maipulation.
Simple regex block
Detailed Documentation:https://rapids.readthedocs.io/projects/nvstrings/en/latest/api.html
'''
%time string_filter_cudf=cdf_file[cdf_file.Country.str.lower().str.contains('^un',regex=True)]
%time string_filter_pandas=pd_file[pd_file.Country.str.lower().str.contains('^un',regex=True)]

CPU times: user 7.72 ms, sys: 0 ns, total: 7.72 ms
Wall time: 7.06 ms
CPU times: user 317 ms, sys: 4.05 ms, total: 321 ms
Wall time: 320 ms


In [22]:
from IPython import get_ipython
get_ipython().magic('reset -sf') 
import gc
gc.collect()

0